In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from gensim.models import Word2Vec
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Bidirectional,Embedding
import re

In [2]:
data = pd.read_csv('data.csv')

print(data.head())

   id                                           consulta          intencion
0   1   ¿Cuál es la fecha límite para el proyecto Final?     fechas_entrega
1   2   ¿Cuáles son los temas a tratar en la clase de...            temario
2   3   ¿Dónde puedo encontrar material de estudio ad...   material_estudio
3   4   ¿Cuál es el horario de consulta del profesor ...           horarios
4   5                     ¿Cómo implemento un árbol AVL?    ejemplos_codigo


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         102 non-null    int64 
 1   consulta   102 non-null    object
 2   intencion  102 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB


In [4]:
data.columns

Index(['id', 'consulta', 'intencion'], dtype='object')

In [5]:
# removiendo el espacio de las columnas
data.columns = data.columns.str.strip()
data['consulta'] = data['consulta'].str.strip()
data['intencion'] = data['intencion'].str.strip()

print(data.values)

[[1 '¿Cuál es la fecha límite para el proyecto Final?' 'fechas_entrega']
 [2 '¿Cuáles son los temas a tratar en la clase de esta semana?'
  'temario']
 [3 '¿Dónde puedo encontrar material de estudio adicional?'
  'material_estudio']
 [4
  '¿Cuál es el horario de consulta del profesor de Programación Avanzada?'
  'horarios']
 [5 '¿Cómo implemento un árbol AVL?' 'ejemplos_codigo']
 [6 '¿Hay alguna práctica programada para el próximo fin de semana?'
  'horarios']
 [7
  '¿Dónde puedo encontrar ejemplos de código para algoritmos de ordenamiento eficientes?'
  'ejemplos_codigo']
 [8 '¿Cuál es el cronograma de exámenes para este semestre?' 'cronograma']
 [10 '¿Cuándo es la entrega de la implementación de árboles rojinegros?'
  'fechas_entrega']
 [11
  '¿Dónde puedo encontrar información sobre la solución del ejercicio de backtracking?'
  'material_estudio']
 [12
  '¿Hay algún repositorio de código disponible para la práctica de pilas y colas en Python?'
  'repositorio']
 [13
  '¿Cómo puedo ha

In [6]:
#sort data by intencion
data = data.sort_values(by=['intencion'], ascending=True)
print(data.head())

    id                                           consulta   intencion
50  52  ¿Dónde puedo obtener una copia del cronograma ...  cronograma
43  45  ¿Dónde puedo encontrar el cronograma de exámenes?  cronograma
28  30  ¿Dónde puedo obtener una copia del cronograma ...  cronograma
27  29  ¿Cuál es el cronograma de entregas para las ta...  cronograma
52  54  ¿Dónde puedo encontrar el cronograma de exámenes?  cronograma


In [7]:
# Eliminando los signos de puntuacion
data['consulta'] = data['consulta'].str.replace(r'[^\w\s]', '', regex=True)

In [8]:
train_txt,test_txt,train_label,test_labels = train_test_split(data['consulta'],data['intencion'],test_size = 0.2)

In [9]:
# Convertir el texto en una lista de palabras
words = []
for text in data['consulta']:
    words += text_to_word_sequence(text)

# Contar el número de palabras únicas
unique_words = set(words)
num_unique_words = len(unique_words)

# Imprimir el número de palabras únicas
print("Número de palabras únicas: ", num_unique_words)

Número de palabras únicas:  211


In [10]:
max_num_words = 350 #el numero de palabras unicas es de 211 entonces agregamos un maximo con holgura
classes = np.unique(data['intencion'])

tokenizer = Tokenizer(num_words=max_num_words)
tokenizer.fit_on_texts(train_txt)
word_index = tokenizer.word_index

word_index

{'de': 1,
 'el': 2,
 'la': 3,
 'para': 4,
 'puedo': 5,
 'es': 6,
 'dónde': 7,
 'cuál': 8,
 'encontrar': 9,
 'del': 10,
 'horario': 11,
 'fecha': 12,
 'los': 13,
 'límite': 14,
 'en': 15,
 'repositorio': 16,
 'pa': 17,
 'cronograma': 18,
 'entrega': 19,
 'temario': 20,
 'las': 21,
 'avanzada': 22,
 'al': 23,
 'este': 24,
 'que': 25,
 'proyecto': 26,
 'cuáles': 27,
 'son': 28,
 'temas': 29,
 'se': 30,
 'quiero': 31,
 'ejemplos': 32,
 'programación': 33,
 'tp': 34,
 'cuatrimestre': 35,
 'código': 36,
 'final': 37,
 'actualizado': 38,
 'con': 39,
 'necesito': 40,
 'específicos': 41,
 'próximo': 42,
 'práctica': 43,
 'cómo': 44,
 'curso': 45,
 'programacion': 46,
 'semana': 47,
 'mes': 48,
 'tutoría': 49,
 'árboles': 50,
 'sesiones': 51,
 'trabajo': 52,
 'consulta': 53,
 'tenes': 54,
 'enlace': 55,
 'presentación': 56,
 'horarios': 57,
 'exámenes': 58,
 'material': 59,
 'estudio': 60,
 'clases': 61,
 'esta': 62,
 'mi': 63,
 'por': 64,
 'implementación': 65,
 'algoritmos': 66,
 'acceder': 67

In [11]:
ls=[]
for c in train_txt:
    ls.append(len(c.split()))   #cada consulta se convierte en una lista de palabras y se cuenta el numero de palabras

maxLen=int(np.percentile(ls, 98))   #se calcula el percentil 98 de la lista de palabras

train_sequences = tokenizer.texts_to_sequences(train_txt)   #convierte las consultas en secuencias de tokens
print("train sequence tokenice:\n",train_sequences)

train_sequences = pad_sequences(train_sequences, maxlen=maxLen, padding='post')     #rellena con ceros las secuencias para que todas tengan la misma longitud dada por el percentil 98
print("train sequence pad sequence:\n",train_sequences)

test_sequences = tokenizer.texts_to_sequences(test_txt)
print("test sequence tokenice:\n",test_sequences)
test_sequences = pad_sequences(test_sequences, maxlen=maxLen, padding='post')
print("test sequence tokenice:\n",test_sequences)

train sequence tokenice:
 [[8, 6, 2, 11, 1, 53, 10, 92, 1, 33, 22], [54, 2, 55, 23, 16, 1, 17], [8, 6, 3, 12, 14, 4, 3, 56, 10, 26, 1, 93, 1, 45], [8, 6, 3, 12, 14, 4, 3, 19, 34, 1, 46, 22], [40, 94, 57, 70, 1, 24, 35], [27, 28, 13, 29, 41, 25, 30, 95, 15, 24, 35], [96, 2, 18, 1, 58, 4, 2, 42, 97, 71, 15, 2, 45, 1, 17], [72, 98, 16, 1, 36, 99, 4, 3, 43, 1, 100, 101, 102, 15, 73], [27, 28, 13, 29, 41, 25, 30, 74, 15, 2, 20, 1, 3, 103, 47], [40, 59, 75, 1, 60], [27, 28, 13, 29, 41, 25, 30, 74, 15, 2, 20, 10, 42, 48], [7, 5, 9, 2, 11, 49, 4, 34], [8, 6, 3, 12, 14, 10, 34], [7, 5, 9, 2, 11, 1, 21, 104, 1, 24, 48], [54, 2, 18], [8, 6, 2, 11, 1, 105, 1, 3, 49, 4, 2, 26, 37], [7, 5, 9, 2, 20, 38, 39, 13, 106, 107, 76], [7, 5, 9, 2, 11, 1, 21, 61, 1, 108], [27, 28, 13, 29, 41, 25, 30, 109, 62, 47], [7, 5, 9, 2, 20, 38, 39, 21, 110, 1, 24, 35], [8, 6, 3, 12, 14, 4, 3, 19, 34, 111], [27, 28, 13, 29, 112, 113, 15, 3, 77, 1, 62, 47], [7, 5, 78, 79, 80, 10, 18, 1, 114], [8, 6, 3, 12, 14, 4, 3, 56, 

In [12]:
label_encoder = LabelEncoder() # codifica las clases en numeros
integer_encoded = label_encoder.fit_transform(classes) # codifica las clases en numeros

print("Clases: ", classes)
print("Clases codificadas: ", integer_encoded)

Clases:  ['cronograma' 'ejemplos_codigo' 'fechas_entrega' 'horarios'
 'material_estudio' 'repositorio' 'temario']
Clases codificadas:  [0 1 2 3 4 5 6]


In [13]:
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

print(integer_encoded)

[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]]


In [14]:
onehot_encoder.fit(integer_encoded)

print("Onehot catgories:",onehot_encoder.categories_) # codifica las clases en numeros

Onehot catgories: [array([0, 1, 2, 3, 4, 5, 6])]


c:\Users\Marcos\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [34]:
# dimesiones del onehot_encoder
num_classes = len(onehot_encoder.categories_[0])
print("Número de clases: ", num_classes)
print("Clases \n", onehot_encoder.categories_[0])

Número de clases:  7
Clases 
 [0 1 2 3 4 5 6]


In [15]:
train_label

3            horarios
88        repositorio
31     fechas_entrega
54     fechas_entrega
90           horarios
           ...       
34           horarios
6     ejemplos_codigo
7          cronograma
64           horarios
91            temario
Name: intencion, Length: 81, dtype: object

In [16]:
train_label_encoded = label_encoder.transform(train_label) # codifica las clases en numeros

train_label_encoded

array([3, 5, 2, 2, 3, 6, 0, 5, 6, 4, 6, 3, 2, 3, 0, 3, 6, 3, 6, 6, 2, 6,
       0, 2, 0, 2, 4, 6, 4, 3, 6, 3, 5, 5, 4, 6, 1, 1, 3, 3, 0, 3, 5, 3,
       1, 2, 5, 5, 3, 0, 4, 5, 2, 3, 4, 4, 5, 0, 3, 4, 4, 2, 2, 2, 5, 2,
       2, 0, 2, 3, 4, 6, 0, 5, 2, 3, 3, 1, 0, 3, 6])

In [17]:
train_label_encoded = train_label_encoded.reshape(len(train_label_encoded), 1)

train_label_encoded

array([[3],
       [5],
       [2],
       [2],
       [3],
       [6],
       [0],
       [5],
       [6],
       [4],
       [6],
       [3],
       [2],
       [3],
       [0],
       [3],
       [6],
       [3],
       [6],
       [6],
       [2],
       [6],
       [0],
       [2],
       [0],
       [2],
       [4],
       [6],
       [4],
       [3],
       [6],
       [3],
       [5],
       [5],
       [4],
       [6],
       [1],
       [1],
       [3],
       [3],
       [0],
       [3],
       [5],
       [3],
       [1],
       [2],
       [5],
       [5],
       [3],
       [0],
       [4],
       [5],
       [2],
       [3],
       [4],
       [4],
       [5],
       [0],
       [3],
       [4],
       [4],
       [2],
       [2],
       [2],
       [5],
       [2],
       [2],
       [0],
       [2],
       [3],
       [4],
       [6],
       [0],
       [5],
       [2],
       [3],
       [3],
       [1],
       [0],
       [3],
       [6]])

In [18]:
train_label = onehot_encoder.transform(train_label_encoded)    # codifica las clases en numeros

train_label

array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0.],
 

In [19]:
#ahora lo mismo pero para los datos de testeo

test_labels_encoded = label_encoder.transform(test_labels)
test_labels_encoded = test_labels_encoded.reshape(len(test_labels_encoded), 1)

print("pre-onehot:\n",test_labels)

test_labels = onehot_encoder.transform(test_labels_encoded)

print("\n\npos-onehot:\n",test_labels)

pre-onehot:
 5            horarios
33     fechas_entrega
78        repositorio
32     fechas_entrega
48           horarios
49           horarios
58           horarios
37            temario
35            temario
77         cronograma
94    ejemplos_codigo
85            temario
30        repositorio
98            temario
43         cronograma
39         cronograma
21    ejemplos_codigo
60         cronograma
69         cronograma
16    ejemplos_codigo
27         cronograma
Name: intencion, dtype: object


pos-onehot:
 [[0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]

In [20]:
words

['dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'del',
 'cronograma',
 'de',
 'prácticas',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'de',
 'exámenes',
 'dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'del',
 'cronograma',
 'de',
 'evaluaciones',
 'para',
 'el',
 'próximo',
 'año',
 'académico',
 'cuál',
 'es',
 'el',
 'cronograma',
 'de',
 'entregas',
 'para',
 'las',
 'tareas',
 'restantes',
 'del',
 'semestre',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'de',
 'exámenes',
 'dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'del',
 'cronograma',
 'de',
 'practicas',
 'de',
 'pa',
 'programación',
 'avanzada',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'de',
 'exámenes',
 'dónde',
 'puedo',
 'encontrar',
 'el',
 'cronograma',
 'actualizado',
 'de',
 'evaluaciones',
 'de',
 'programacion',
 'avanzada',
 'cuál',
 'es',
 'el',
 'cronograma',
 'de',
 'pa',
 'de',
 'este',
 'cuatrimestre',
 'dónde',
 'puedo',
 'obtener',
 'una',
 'copia',
 'de

In [21]:
w2v_model = Word2Vec(sentences=[words], vector_size=100, window=5, min_count=1, workers=4)
"""
    Parametros:
    sentences: lista de lista de palabras
    size: dimension del vector (posible parametro a modificar 100-300)
    window: tamaño de la ventana de palabras (ventana: palabras que se encuentran a los lados de la palabra objetivo)
    min_count: minimo de veces que debe aparecer una palabra para ser considerada (es un dataset chico asi que se deja en 1)
    workers: numero de hilos de ejecucion
"""


'\n    Parametros:\n    sentences: lista de lista de palabras\n    size: dimension del vector (posible parametro a modificar 100-300)\n    window: tamaño de la ventana de palabras (ventana: palabras que se encuentran a los lados de la palabra objetivo)\n    min_count: minimo de veces que debe aparecer una palabra para ser considerada (es un dataset chico asi que se deja en 1)\n    workers: numero de hilos de ejecucion\n'

In [22]:
# mostrar las dimensiones de la matriz de embeddings
print("Dimensiones de la matriz de embeddings: ", w2v_model.wv.vectors.shape)

Dimensiones de la matriz de embeddings:  (211, 100)


In [23]:
# mostramos los vectores de las palabras
print(w2v_model.wv.vectors[1])

[-0.01076306  0.00718829  0.00408915  0.00642215  0.00866755 -0.01304841
  0.00412582  0.0141335  -0.00699785 -0.00908607 -0.00165412 -0.0137612
 -0.00592462  0.0096036   0.00455162  0.00379571  0.00856021  0.00500713
 -0.00490827 -0.00973071  0.00302717 -0.0022888   0.01104611 -0.01068563
  0.00604263  0.00342569 -0.00686546  0.00341924 -0.00484632  0.00686655
  0.01391384 -0.00436521  0.00265058 -0.01018085  0.00103897  0.00892116
  0.00986585  0.00460085  0.00862124  0.00404157  0.00941136 -0.01202924
 -0.01113608 -0.00191462  0.00051403  0.00865595  0.0025814  -0.00248957
  0.00399516  0.0031886   0.00960894 -0.0115593   0.00103191  0.00302574
 -0.00141472  0.00918643  0.01154194  0.00586984 -0.00407728  0.00939944
 -0.00758446  0.00232315 -0.00297465 -0.00592903  0.00166015  0.00957097
  0.00843098 -0.00079913  0.00111493  0.01165282 -0.00678038 -0.00704475
  0.00908658  0.00675251  0.00394422 -0.00549796 -0.00782748 -0.00031667
  0.00254652 -0.00385612 -0.01217128  0.0049877   0.

In [24]:
# mostramos los embeddings de la palabra "hola"
print(w2v_model.wv['dónde'])

[-9.2231408e-03  4.7834730e-03 -8.2930280e-03 -1.6092287e-03
  5.0338195e-03 -1.0380825e-03  4.1080848e-03  1.0306911e-02
 -8.2265800e-03  4.3195765e-03 -7.4328827e-03 -7.7517319e-04
 -8.4470846e-03  4.5720413e-03 -3.8525849e-03 -7.4047204e-03
 -1.4502687e-03  3.1906622e-03  4.7205286e-03 -1.3973443e-02
  1.4518070e-03 -6.2836739e-03  1.0256744e-02  8.5055986e-03
 -3.2795351e-03  1.2684301e-03 -1.1400800e-03  4.3563684e-03
 -1.1708008e-02  6.6899665e-04  1.0694679e-02  2.4324651e-03
  4.3680011e-03 -1.3705671e-02  5.6707393e-03 -1.5508276e-04
  1.3191091e-04  1.8351773e-03 -1.5997282e-03 -3.9339345e-03
  3.1392740e-03 -1.1795669e-02 -1.1673977e-02  7.2277579e-03
  9.7305411e-03 -6.1917901e-03  6.9202455e-05 -7.9927000e-04
  7.1327738e-03 -5.8215763e-03  5.8255824e-03  2.3192461e-03
  1.0962043e-02 -4.7695665e-03 -1.5980708e-03 -6.6363574e-03
 -7.1133762e-03 -9.8364791e-03 -4.3229843e-03 -4.8678573e-03
  5.6163482e-03 -6.9436939e-03  4.2905966e-03 -1.2481523e-04
 -5.3811269e-03  3.26550

In [25]:
# mostramos el tamaño del vocabulario
print("Tamaño del vocabulario: ", len(w2v_model.wv))

Tamaño del vocabulario:  211


In [26]:
modelo_LSTM = tf.keras.Sequential()

modelo_LSTM.add(Embedding(input_dim=num_unique_words+1, output_dim=100, input_length=maxLen, trainable=True, weights=[w2v_model.wv.vectors]))
"""
    Parametros:
    input_dim: tamaño del vocabulario (numero de palabras unicas)
    output_dim: tamaño del vector de salida (dimension del vector)
    input_length: longitud de la secuencia de entrada (numero de palabras en cada consulta)

"""
modelo_LSTM.add(Bidirectional(LSTM(128  , return_sequences=True, dropout=0.1, recurrent_dropout=0.1), 'concat'))
"""
    Parametros:
    units: dimension de la salida del LSTM (numero de neuronas)
    return_sequences: si la salida es una secuencia o un vector (True: secuencia, False: vector)
    dropout: porcentaje de neuronas que se desactivan en cada iteracion (evita el overfitting)
    recurrent_dropout: porcentaje de neuronas que se desactivan en cada iteracion de la capa recurrente (evita el overfitting)

    concat: concatena las salidas de las capas recurrentes (bidireccional)
"""
# modelo_LSTM.add(LSTM(128, dropout=0.1, recurrent_dropout=0.1, return_sequences=False))
"""
    esta capa cumple la funcion de reducir la dimension de la salida de la capa anterior
    esto se debe a que la capa anterior devuelve una secuencia de vectores y la siguiente capa espera un vector
"""
modelo_LSTM.add(Dense(64, activation='relu'))
"""
    Parametros:
    units: dimension de la salida de la capa, es 64 porque es la mitad de la salida de la capa anterior
    activation: funcion de activacion
"""
modelo_LSTM.add(Dense(num_classes, activation='softmax'))
"""
    Parametros:
    units: dimension de la salida de la capa, es num_classes porque es el numero de clases
    activation: funcion de activacion
"""
modelo_LSTM.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

ValueError: Layer embedding weight shape (212, 100) is not compatible with provided weight shape (211, 100).